# Precessamento dos dados 

Após importar os dados, chegou a hora de verificarmos se estão adequados para análise.
As tarefas são as seguintes:

- [x] Estudar o dicionário de dados (arquivo METODOLOGIA.docx) e marcar o que for importante 
- [ ] Verificar o tipo dos dados
- [ ] Verificar a integridade dos dados:
  - [ ] Todas as colunas correspondem ao descrito no dicionário de dados?
  - [ ] Há duplicatas?
  - [ ] Há dados ausentes? São significativos?
  - [ ] Há dados incorretos, mal formatados, etc.?

[Link da documentação para trabalhar a consistencia das datas](https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html)

## Conferindo a integridade dos dados

Na etapa anterior, [Coleta e Preparação dos Dados]("./03_coleta_e_preparação_dos_dados.ipynb"), utilizamos o seguinte script para importar os dados e transformar o nome dos arquivos em um padrão adequado: 

In [1]:
import os
import re
import shutil
import pandas as pd

input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

MONTHS = {
    "1": "Janeiro",
    "2": "Fevereiro",
    "3": "Março",
    "4": "Abril",
    "5": "Maio",
    "6": "Junho",
    "7": "Julho",
    "8": "Agosto",
    "9": "Setembro",
    "10": "Outubro",
    "11": "Novembro",
    "12": "Dezembro"
}

pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")


def process_file(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith(".xls"):
            match = pattern.search(filename)
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                new_name = pattern.sub(f"{name_month}", filename)

                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
            for file in file_path_generator]

df = pd.concat(datasets, axis=0, ignore_index=True)

pd.set_option("display.max_columns", None)


/tmp/ipykernel_21263/902107909.py:49: DtypeWarning: Columns (31,32,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_21263/902107909.py:49: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_21263/902107909.py:49: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")


Observe que, ao realizar a importação, recebemos um aviso `DtypeWarning`. Iremos começar verificar a consistência e integridade dos dados compreendendo a razão pela qual este aviso ocorre e checar como podemos processar os dados adequadamente para torná-los úteis para análise. 

### Solucionando os problemas de tipo dos dados

A saída `DtypeWarning` do comando de importação nos avisa que o `pandas` não conseguiu identificar o tipo de algumas colunas do dataset e, por isso, teve que inferir um tipo. Em outras palavras, o `pandas` tenta determinar os tipos de cada coluna; uma operação que exige muita memória.  

Podemos silenciar esses avisos basicamente de duas formas: A primeira é utilizando o parâmetro `dtypes` e passando `object` como valor.`Object` [é a classe base de onde derivam toda as classes em Python](https://docs.python.org/pt-br/3/library/functions.html?highlight=object#object), ou seja, `object` pode conter qualquer objeto Python. Isso fará com que o `pandas` defina o tipo `object` para todos os dados. Esse tipo já é aplicado para tipos de dados mistos ou que o `pandas` não consegue inferir. [No entanto, essa medida deve ser evitado na medida do possível](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes) (para desempenho e interoperabilidade com outras bibliotecas e métodos).

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                dtypes=object) 
                for file in file_path_genarator]
```

A outra forma é usar o parâmetro `low-memory=False`, como sugere o próprio aviso: `DtypeWarning: Columns (...,...) have mixed types. Specify dtype option on import or set low_memory=False`.

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                low-memory=True) 
                for file in file_path_genarator]
```
 Porém, esse parâmetro, embora aparente funcionar, é considerado obsoleto e pode levar a erros silenciosos que não são documentados. [Existe uma discussão que se arrasta desde 2014 sobre se este parâmetro deve ou não ser melhor documentado ou retirado da biblioteca](https://github.com/pandas-dev/pandas/issues/5888). O fato é que sua utilização não é muito comum apesar do aviso indicar seu uso.

De toda forma, nenhuma das abordagens resolvem o problema de fato. O `pandas` continuará usando muito recurso para adivinhar os tipos. 

Portanto, a abordagem mais indicada nesses casos é fornecer os tipos manualmente. 

Antes de partir para esta tarefa, vamos checar o estado do DataFrame:


In [2]:
df.shape

(119158, 54)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ANO_BO                    119158 non-null  int64  
 1   NUM_BO                    119158 non-null  int64  
 2   NUMERO_BOLETIM            119158 non-null  object 
 3   BO_INICIADO               119158 non-null  object 
 4   BO_EMITIDO                119158 non-null  object 
 5   DATAOCORRENCIA            119158 non-null  object 
 6   HORAOCORRENCIA            110412 non-null  object 
 7   PERIDOOCORRENCIA          119158 non-null  object 
 8   DATACOMUNICACAO           119158 non-null  object 
 9   DATAELABORACAO            119158 non-null  object 
 10  BO_AUTORIA                119158 non-null  object 
 11  FLAGRANTE                 119158 non-null  object 
 12  NUMERO_BOLETIM_PRINCIPAL  37625 non-null   object 
 13  LOGRADOURO                111762 non-null  o

Podemos ver que, com exceção de alguns tipos `float64`, o pandas inferiu `object` em todo o conjunto de dados. Trabalhar com os tipos corretos no DataFrame é extremamente importante para que possamos garantir:
- Precisão nos dados
- Economia de memória
- Desempenho
- Manipulação dos dados mais eficaz
- Prevenção de erros
- Consistência. 


Vamos olhar uma amostra aleatória dos dados para termos uma noção da tipagem que precisará ser adequada.

In [6]:
df.sample(5)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
43625,2023,3517,3517/2023,13/06/2023 17:31:02,13/06/2023 17:31:02,13/06/2023,06:17,PELA MANHÃ,13/06/2023,13/06/2023 17:31:02,Desconhecida,Não,NaN,AVENIDA ELDORADO,10.0,JD DA ESTAÇÃO,ITAQUAQUECETUBA,SP,"-23,479395214","-46,3667920569999",Via pública,NaN,APRECIAÇÃO DO DELEGADO TITULAR,DEL.POL.ITAQUAQUECETUBA,DEL.POL.ITAQUAQUECETUBA,Localização e/ou Devolução,Localização/Apreensão e Entrega de veículo,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RNR0F11,MG,BELO HORIZONTE,Branco,VW/T CROSS HL TSI AE,2021.0,2021.0,AUTOMOVEL,1.0,Apple
95209,2023,1314,1314/2023,02/03/2023 09:42:55,02/03/2023 09:42:55,27/02/2023,20:40,A NOITE,02/03/2023,02/03/2023 09:42:55,Desconhecida,Não,NaN,Rua Gonçalo Francisco Xavier,41.0,Jardim Cidade Pirituba,S.PAULO,SP,"-23,4771828","-46,7194106",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,33º D.P. PIRITUBA,87º D.P. V. P. BARRETO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEY2C20,SP,S.PAULO,Prata,I/VOLVO XC60 3.0T AWD,2009.0,2010.0,CAMIONETA,1.0,Apple
87119,2023,615836,615836/2023,22/04/2023 09:42:39,22/04/2023 09:42:44,19/04/2023,22:00,A NOITE,20/04/2023,22/04/2023 09:42:39,Desconhecida,Não,NaN,Rua Joaquim Antunes,501.0,Pinheiros,S.PAULO,SP,"-23,5643745","-46,681808",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,14º D.P. PINHEIROS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
111608,2023,383,383/2023,22/03/2023 18:52:44,22/03/2023 18:52:44,22/03/2023,18:10,A NOITE,22/03/2023,22/03/2023 18:52:44,Desconhecida,Não,NaN,RUA DARIO FERREIRA MARTINS,100.0,VILA CURUCA,S.PAULO,SP,"-23,4924346719999","-46,429876796",Via pública,NaN,BO PARA INVESTIGAÇÃO,22º D.P. SAO MIGUEL PTA,22º D.P. SAO MIGUEL PTA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Motorola
75809,2023,98165,98165/2023,04/04/2023 23:48:56,04/04/2023 23:49:14,03/04/2023,NaN,A NOITE,04/04/2023,04/04/2023 23:48:56,Desconhecida,Não,NaN,RUA MANOEL FERREIRA PIRES,320.0,VILA FORMOSA,S.PAULO,SP,"-23,581870592","-46,5266242289999",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 1,41º D.P. VILA RICA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple


Para fornecer ao `pandas` as informações corretas sobre os tipos dos dados, precisamos ter certeza de quais tipos eles são. Nesse ponto o dicionário de dados é fundamental . Como vimos no início da etapa anterior ([Coleta e Preparação dos Dados]("./03_coleta_e_preparação_dos_dados.ipynb")) A Secretaria de Segurança fornece um arquivo de texto intitulado "METODOLOGIA" contendo informações sobre o conjunto de dados. 

Como uma tarefa complementar ao nosso objetivo nesta etapa, para facilitar a comparação entre o dicionário e o DataFrame, eu exportei a saída da propriedade `dtypes` e copiei a tabela do documento "METODOLOGIA" para uma planilha a fim de:

1. Checar se o DataFrame contém todas as variáveis presentes no dicionário de dados;
2. Identificar os tipos dos dados e atribuí-los adequadamente ao DataFrame.

O código para exportação segue abaixo. A planilha se encontra no diretório `data` e, como poderá ser observado, fiz uma busca vertical (VLOOKUP ou PROCV) para cruzar as colunas que se referem às características do conjunto de dados. 

In [8]:
t = df.dtypes.reset_index()
t.columns = ["Variáveis", "Tipos"]
t.to_excel("./data/dtypes_out.xlsx", index=False)

No entanto, ao analisar o dicionário de dados, cheguei à conclusão de que ele oferece pouca ajuda para compreender o significado das variáveis além do que é possível inferir em uma comparação direta óbvia. Os motivos são destacados a seguir:

1. O DataFrame possui 54 variáveis enquanto o dicionário indica 62;
2. O dicionário não informa os tipos dos dados;
3. As descrições das variáveis são vagas e não fornecem mais informações além da repetição do nome da variável;
4. O nome das variáveis no dicionário são diferentes dos nomes no DataFrame, mas algumas se referem à mesma coisa. Por exemplo: `BOLETIM_EMITIDO` não consta no dicionário, mas é equivalente à "DATAHORA_IMPRESSAO_BO".


O dicionário de dados informa que, várias linhas podem se referir ao mesmo boletim. Portanto, para conclusões quanto as quantidades de ocorrências é necessária exclusão das duplicidades das seguintes variáveis:
    • NOME_DELEGACIA
    • ANO_BO
    • NUM_BO


Quando comparamos o dicionario de dados e as variáveis do DataFrame, notamos duas coisas:

1. O DataFrame possui 54 colunas enquanto o dicionário indica 62.
2. Algumas variáveis estão diferentes mas se referem a mesma coisa. Por exemplo: `BOLETIM_EMITIDO` não consta no dicionário, mas é equivalente à "DATAHORA_IMPRESSAO_BO". Este problema se repete com outras variáveis também.
3. Os valores de `LATITUDE` e `LONGITUDE` não são interpretados como `float`, mas `object`. Além disso, alguns possuem um espaço em branco tanto à esquerda quanto à direita. Será necessário limpar estes espaços, substituir a "," por "." e converter o dados para float.
4. A única coluna que se refere a valores do tipo int e que pode ser utilizada para cálculo é "QUANT_CELULAR", que se refere a quantidade de dispositivos roubados da vítima, também é interpretado como `object` e possui valores nulos. Portanto, precisamos convertê-lo para o tipo adequado que suporte valores ausentes.
5. Há uma inconsistência na formatação das strings. As variáveis relacionadas ao local do roubo ou endereço como `LOGRADOURO` e `BAIRRO`, por exemplo, traz caracteres minúsculos e maiúsculo. Para garantir a consistência desses dados, vou converter todas as strings para maiúsculo.
6. Há muitos dados ausentes. Será necessário definir como lidar com esses valores identificando-os e verificando sua natureza. 

A conclusão que eu chego, por enquanto, é que o dicionário de dados disponibilizado pela Secretaria de Segurança pouco nos ajuda a entender o significado das variáveis para além do que é possível inferir em uma comparação direta óbvia. 

Caso não limpemos, mesmo utilizando o tipo Float64Dtype() obtemos o ValueError: Unable to parse string "-22,9181399". Se passarmos para o dicionário DTYPES a função para o campo referente a essas colunas, obtemos o TypeError: Cannot interpret '' as a data type, ou seja, precisamos passar para o parâmetro `converters` que irá cuidar da conversão. 


Em uma estrutura tabular como o DataFrame, o conceito de `axis` (eixos) refere-se às dimensões da estrutura. Se pensarmos na estrutua tabular como um plano cartesiano, `axis=0` estaria para o eixo y assim como `axis=1` para o eixo x. Isso quer dizer que uma operação realizada no `axis=0` será aplicada ao longo da dimensão das linhas, isto é, de cima para baixo. Por outro lado, uma operação realizada no `axis=1` é aplicada na dimensão das colunas, da esquerda para direita. 

Quando determinamos o parâmetro `axis=0` no métodos `pandas.concat` estamos especificando que a concatenação será feita verticalmente, ou seja, mantemos o eixo das colunas e adicionamos linhas ao dataframe.

Utilizamos o parâmetro `ignore_index=True` para ignorar os índices dos datasets que serão concatenados, isso faz com que o `pandas` gere uma sequencia contínua de indices. Se passássemos `False`, o primeiro dataset seria indexado de 1-n, mas o segundo também e assim por diante, o que acarretaria em indices duplicados. 

Para termos uma noção do estado dos dados podemos chamar o método `info()`. A saída desse método deve fornecer as seguintes informações:
Column: Esta coluna indica os nomes das colunas presentes no DataFrame.

Non-Null Count: Esta coluna mostra a contagem de valores não nulos em cada coluna. Em outras palavras, é o número de entradas que possuem valores válidos para aquela coluna. Por exemplo, na primeira coluna "ANO_BO", existem 119158 valores não nulos.

Dtype: Esta coluna informa o tipo de dados (data type) presente em cada coluna. Por exemplo, "int64" indica que a coluna contém valores inteiros de 64 bits, e "object" indica valores de tipo objeto, que geralmente são strings (texto) ou valores mistos.

Para vermos quantas colunas e quantas linhas o dataframe possui, usamos a propriedade `shape`:

In [ ]:
df.shape

O dicionário de dados indica que as tabelas possuem 62 colunas, contudo, o dataframe possui 54 colunas. 


In [ ]:
data1 = {"A": [1, 2, 3], "B": [4, 5, 6]}

data2 = {"C": [7, 8, 9], "D": [10, 11, 12], "E": [13, 14, 15]}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

t = pd.concat([df1, df2], axis=0, ignore_index=True)



In [ ]:
t.head()

Minha primeira dúvida era entender se havia a possibilidade do pandas ter ignorando alguma coluna presente em um ou mais datasets e ausente em outros. Mas não. Como podemos ver no exemplo acima, quando temos dois datasets com quantidades e/ou nomes diferentes, no momento da concatenação é feita a união das colunas e os valores presentes em uma e ausentes em outra são preenchidos por `NaN` (nulo).

Isso elimina a hipotese de termos uma eventual perda de características no momento da concatenação dos dataframes. Quando comparamos o dicionario de dados e as variáveis do dataframe, notamos que algumas variáveis estão diferentes mas se referem a mesma coisa. Por exemplo: `BOLETIM_EMITIDO` não consta no dicionário, mas é equivalente à "DATAHORA_IMPRESSAO_BO". A conclusão que eu chego, por enquanto, é que o dicionário de dados disponibilizado pela Secretaria de Segurança pouco nos ajuda a entender o significado das variáveis para além do que é possível inferir em uma comparação direta óbvia. 

Para sabermos quais colunas o dicionário de dados apresenta a mais, fiz uma planilha e 

In [19]:
df.sample(5)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
53608,2023,1007604,1007604/2023,27/06/2023 15:50:19,27/06/2023 15:50:26,26/06/2023,08:30,PELA MANHÃ,27/06/2023,27/06/2023 15:50:19,Desconhecida,Não,NaN,RUA SERRA DO GRAO MOGOL,111.0,JARDIM HELENA,S.PAULO,SP,"-23,4928915840974","-46,405438794374",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,59º D.P. JARDIM DOS IPES,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
112601,2023,1607,1607/2023,23/03/2023 18:29:23,23/03/2023 18:29:23,23/03/2023,03:00,DE MADRUGADA,23/03/2023,23/03/2023 18:29:23,Desconhecida,Não,1606/2023 - 30418,RODOVIA PRESIDENTE DUTRA (BR 116),178.0,ÁREA RURAL,JACAREI,SP,"-23,307353056","-46,044865409",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,05º D.P. OSASCO,02º D.P. JACAREI,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - CARGA,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IZY8C73,SC,PESCARIA BRAVA,Preta,SR/LIBRELATO CACAENCR 3E,2020.0,2020.0,SEMI-REBOQUE,1.0,Motorola
2459,2023,23826,23826/2023,04/01/2023 09:18:20,04/01/2023 09:18:25,31/12/2022,22:30,A NOITE,03/01/2023,04/01/2023 09:18:20,Desconhecida,Não,NaN,RUA CÂNDIDO RODRIGUES,75.0,VILA JUNQUEIRA,S.ANDRE,SP,"-23,6901827555488","-46,5102629186103",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,06º D.P. SANTO ANDRÉ,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
65124,2023,578,578/2023,18/02/2023 11:27:49,18/02/2023 11:27:49,17/02/2023,20:50,A NOITE,18/02/2023,18/02/2023 11:27:49,Conhecida,Não,NaN,ACESSO RODOVIA ANHANGUERA,104.0,CHACARA TRES MARIAS,CAMPINAS,SP,"-22,8614882529999","-47,141175914",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,PLANTÃO - 01º D.P. JUNDIAI,08º D.P. CAMPINAS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EQP2389,SP,ITUPEVA,Amarelo,HONDA/CB 300R,2009.0,2010.0,MOTOCICLO,1.0,Samsung
13520,2023,120920,120920/2023,02/05/2023 09:44:11,02/05/2023 09:44:11,27/04/2023,NaN,A NOITE,28/04/2023,02/05/2023 09:44:11,Desconhecida,Não,119033/2023 - 900021,AVENIDA BRIGADEIRO FARIA LIMA,1087.0,ITAIM BIBI,S.PAULO,SP,"-23,5970653699999","-46,679452494",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 1,15º D.P. DR. LUC. H BEIGUELMAN,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple


### Conversão dos tipos


O dataset apresenta algumas inconsistências. A primeria delas é referente ao tipo dos dados.  

In [2]:
import os
import re
import shutil
import pandas as pd

In [57]:
"""
Tente limpar o dado e convertê-lo para float.
Caso não for possível a conversão e o dado não for None transforme em maiúsculo 


"""
def clean_coordinate_value(value):
    if isinstance(value, str):
        try:
            clean_value = value.strip().replace(",", ".")
            return float(clean_value)
        except ValueError:
            return None 
        
    return None


def to_upper(c):
    if type(c) == str:
        try:
            return c.upper()
        except ValueError:
            return  None

    return c 


def process_file(input_dir, output_dir):

    MONTHS = {
        "1": "Janeiro",
        "2": "Fevereiro",
        "3": "Março",
        "4": "Abril",
        "5": "Maio",
        "6": "Junho",
        "7": "Julho",
        "8": "Agosto",
        "9": "Setembro",
        "10": "Outubro",
        "11": "Novembro",
        "12": "Dezembro"
    }

    for filename in os.listdir(input_dir):
        # Verificar se o arquivo é um arquivo xls
        if filename.endswith(".xls"):
            # Pesquisar o padrão
            match = pattern.search(filename)
            # Se verdadeiro, capturar o dígito da string e utilizá-lo para obter o valor da chave do dict `meses`
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                # Novo nome do arquivo após substituição
                new_name = pattern.sub(f"{name_month}", filename)

                # Caminhos completos dos arquivos antigo e novo
                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


In [58]:
DTYPES = {
    "ANO_BO": pd.StringDtype(),
    "ANO_FABRICACAO": pd.StringDtype(),
    "ANO_MODELO": pd.StringDtype(),
    "BO_AUTORIA": pd.StringDtype(),
    "CIDADE": pd.StringDtype(),
    "CIDADE_VEICULO": pd.StringDtype(),
    "CORCUTIS": None,
    "DELEGACIA_CIRCUNSCRICAO": pd.StringDtype(),
    "DELEGACIA_NOME": pd.StringDtype(),
    "DESCR_COR_VEICULO": pd.StringDtype(),
    "DESCR_MARCA_VEICULO": pd.StringDtype(),
    "DESCR_TIPO_VEICULO": pd.StringDtype(),
    "DESCRICAOLOCAL": pd.StringDtype(),
    "DESDOBRAMENTO": pd.StringDtype(),
    "ESPECIE": pd.StringDtype(),
    "ESTADOCIVIL": pd.StringDtype(),
    "EXAME": None,
    "FLAGRANTE": pd.StringDtype(),
    "GRAUINSTRUCAO": pd.StringDtype(),
    "IDADE": pd.Int64Dtype(),
    "LOGRADOURO": pd.StringDtype(),
    "MARCA_CELULAR": pd.StringDtype(),
    "NACIONALIDADE": pd.StringDtype(),
    "NATURALIDADE": pd.StringDtype(),
    "NATUREZAVINCULADA": pd.StringDtype(),
    "NUM_BO": pd.StringDtype(),
    "NUMERO": pd.StringDtype(),
    "NUMERO_BOLETIM": pd.StringDtype(),
    "NUMERO_BOLETIM_PRINCIPAL": pd.StringDtype(),
    "PARENTESCO": pd.StringDtype(),
    "PERIDOOCORRENCIA": pd.StringDtype(),
    "PLACA_VEICULO": pd.StringDtype(),
    "PROFISSAO": pd.StringDtype(),
    "QUANT_CELULAR": pd.Int64Dtype(),
    "RELACIONAMENTO": pd.StringDtype(),
    "RUBRICA": pd.StringDtype(),
    "SEXO": pd.StringDtype(),
    "SOLUCAO": pd.StringDtype(),
    "STATUS": pd.StringDtype(),
    "TIPOPESSOA": pd.StringDtype(),
    "TIPOVINCULO": pd.StringDtype(),
    "UF": pd.StringDtype(),
    "UF_VEICULO": pd.StringDtype(),
    "VITIMAFATAL": pd.StringDtype(),
}

DATE_TYPES = [
    "BO_EMITIDO",
    "BO_INICIADO",
    "DATACOMUNICACAO",
    "DATAELABORACAO",
    "DATANASCIMENTO",
    "DATAOCORRENCIA",
]

DATE_FORMAT = {
    "BO_EMITIDO": "%d/%m/%Y %H:%M:%S",
    "BO_INICIADO": "%d/%m/%Y %H:%M:%S",
    "DATACOMUNICACAO": "%d/%m/%Y",
    "DATAELABORACAO": "%d/%m/%Y %H:%M:%S",
    "DATANASCIMENTO": "%d/%m/%Y",
    "DATAOCORRENCIA": "%d/%m/%Y",
    "HORAOCORRENCIA": "%H:%M",
}

CONVERTERS = {
    "LATITUDE": clean_coordinate_value,
    "LONGITUDE": clean_coordinate_value,
}


input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)


pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")

file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t',
                        encoding="UTF-16 LE",
                        dtype=DTYPES,
                        parse_dates=DATE_TYPES,
                        date_format=DATE_FORMAT,
                        converters=CONVERTERS)
            for file in file_path_generator]

df_temp = pd.concat(datasets, axis=0, ignore_index=True)

for c in df_temp.columns:
    df_temp[c] = df_temp[c].apply(to_upper)

In [59]:
df_temp.sample(10)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
15446,2023,687055,687055/2023,2023-05-04 15:01:35,2023-05-04 15:01:42,2023-05-03,21:30,A NOITE,2023-05-04,2023-05-04 15:01:35,DESCONHECIDA,NÃO,<NA>,AVENIDA 23 DE MAIO,300,VILA MARIANA,S.PAULO,SP,-23.574850,-46.641475,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,36º D.P. VILA MARIANA,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,SAMSUNG
50245,2023,3884,3884/2023,2023-06-22 14:41:04,2023-06-22 14:41:04,2023-06-17,17:00,A TARDE,2023-06-18,2023-06-22 14:41:04,DESCONHECIDA,NÃO,3848/2023 - 20113,RUA MANOEL FERNANDES LEÃO,316,JARDIM PAULISTANO (ZONA NORTE),S.PAULO,SP,-23.442067,-46.709411,VIA PÚBLICA,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,13º D.P. CASA VERDE,74º D.P. PARADA TAIPAS,LOCALIZAÇÃO E/OU DEVOLUÇÃO,LOCALIZAÇÃO/APREENSÃO E ENTREGA DE VEÍCULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,BDB3A80,SP,S.PAULO,PRATA,I/PEUGEOT 308 ALLURE,2012,2013,AUTOMOVEL,1,XIAOMI
1308,2023,3655,3655/2023,2023-01-02 20:13:19,2023-01-02 20:13:19,2023-01-01,11:00,PELA MANHÃ,2023-01-02,2023-01-02 20:13:19,DESCONHECIDA,NÃO,<NA>,AVENIDA MARJORI PRADO,1100,BALNEÁRIO PRAIA PERNAMBUCO,GUARUJA,SP,NaN,NaN,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 3,DEL.POL.GUARUJA,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - TRANSEUNTE,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,MOTOROLA
101737,2023,1288,1288/2023,2023-03-10 13:32:04,2023-03-10 13:32:04,2023-02-22,11:45,PELA MANHÃ,2023-02-22,2023-03-10 13:32:04,CONHECIDA,SIM,981/2023 - 10362,RUA ARRASTÃO,10,PARQUE SANTA MADALENA,S.PAULO,SP,-23.609327,-46.515673,VIA PÚBLICA,NaN,BO PARA FLAGRANTE,69º D.P. TEOTONIO VILELA,70º D.P. VILA EMA,LOCALIZAÇÃO E/OU DEVOLUÇÃO,LOCALIZAÇÃO/APREENSÃO E ENTREGA DE OBJETO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,XIAOMI
109689,2023,1805,1805/2023,2023-03-20 18:37:44,2023-03-20 18:37:44,2023-02-16,20:30,A NOITE,2023-02-17,2023-03-20 18:37:44,DESCONHECIDA,NÃO,1804/2023 - 10356,RUA KATIA LANNER,85,JARAGUA,S.PAULO,SP,-23.451973,-46.751252,VIA PÚBLICA,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,89º D.P. JARDIM TABOAO,46º D.P. PERUS,ATO INFRACIONAL,A.I.-EXTORSÃO (ART. 158),<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE
6991,2023,110,110/2023,2023-01-10 10:25:30,2023-01-10 10:25:30,2023-01-09,21:00,A NOITE,2023-01-10,2023-01-10 10:25:30,DESCONHECIDA,NÃO,<NA>,RUA DOUTOR HENRIQUE CALDERAZZO,373,PARAÍSO,S.ANDRE,SP,-23.670420,-46.533692,VIA PÚBLICA,NaN,APRECIAÇÃO DO DELEGADO TITULAR,01º D.P. SANTO ANDRÉ,01º D.P. SANTO ANDRÉ,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - VEICULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,GJK0J86,SP,S.ANDRE,PRATA,CHEV/TRACKER T A,2022,2022,AUTOMOVEL,1,SAMSUNG
90065,2023,1852,1852/2023,2023-04-25 23:46:25,2023-04-25 23:46:25,2023-04-25,11:20,PELA MANHÃ,2023-04-25,2023-04-25 23:46:25,CONHECIDA,NÃO,<NA>,RUA MOISES ALEXANDRINO DA SILVA,14,BRASILANDIA,S.PAULO,SP,-23.457374,-46.700594,VIA PÚBLICA,NaN,ENCAM FEBEM/V.INFAN.J

In [48]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ANO_BO                    119158 non-null  string        
 1   NUM_BO                    119158 non-null  string        
 2   NUMERO_BOLETIM            119158 non-null  string        
 3   BO_INICIADO               119158 non-null  datetime64[ns]
 4   BO_EMITIDO                119158 non-null  datetime64[ns]
 5   DATAOCORRENCIA            119158 non-null  datetime64[ns]
 6   HORAOCORRENCIA            110412 non-null  object        
 7   PERIDOOCORRENCIA          119158 non-null  string        
 8   DATACOMUNICACAO           119158 non-null  datetime64[ns]
 9   DATAELABORACAO            119158 non-null  datetime64[ns]
 10  BO_AUTORIA                119158 non-null  string        
 11  FLAGRANTE                 119158 non-null  string        
 12  NU

## Limpeza dos dado

---

## Referencias